In [6]:
import requests
from bs4 import BeautifulSoup
from dateutil import parser
import datetime
# from urlparse import urljoin

In [46]:
def cli_green(fn):
    def wrapped(*args, **kwargs):
        return "\033[92m"+fn()
    return wrapped

def scrape_topic_page(topic_number):
    if not isinstance(topic_number, (basestring, int)):
        raise ValueError("Topic number has to be int or basestring")
    soup = BeautifulSoup(
        requests.get(
            'http://netwars.pl/temat/{0!s}'.format(topic_number)
        ).text
    )
    return soup 

from bs4 import BeautifulSoup 
import requests

class NwTopic(object):
    
    NW_URL = 'http://netwars.pl/temat/'
    
    def __init__(self, topic_raw_html, topic_number):
        
        self.soup = BeautifulSoup(topic_raw_html)
        self.topic_number = topic_number
        
        if 'Nie znaleziono tematu' in soup:
            raise ValueError('topic does not exist')
                     
                    
    @classmethod
    def from_url(cls, topic_number):
        r = requests.get(cls.NW_URL+str(topic_number))    
        if r.status_code != 200:
            raise ValueError('failed on get-request')
        return cls(r.text, topic_number)

    @classmethod
    def from_html(cls, topic_raw_html, topic_number):
        """
        To use with already scraped data, for perfromance reasons
        """
        return cls(topic_raw_html, topic_number)
    
    @property
    def posts(self):
        ix = "topic_{0!s}".format(self.topic_number)
        post_list = soup.findAll("div",{"id":ix})
        return post_list
#         if len(post_list) != 1:
#             raise ValueError('Cos tu jest nie tak')
#         return post_list

    
    
class NW(object):
    BASE_URL = 'http://netwars.pl/'
    OT_FORUM_NUMBER = '4'
    
    def __repr__(self):
        return '%s(%r)'%(self.__class__, self.username)
     
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.logged_in = False

    def login(self):
        payload = {
            'tnick':self.username,
            'tpass':self.password
        }
        
        nwsession = requests.session()
        nwsession.post(urljoin(self.BASE_URL,'login'), payload)
        self.nwsession = nwsession
        self.logged_in = True
    
    def logout(self):
        self.nwsession.post(urljoin(self.BASE_URL,'logout'))
        self.logged_in = False
        
    def topics(self):
        forum = self.nwsession.get(
            self.BASE_URL+'forum/'+self.OT_FORUM_NUMBER).text
        soup = BeautifulSoup(forum)
        return [topic.text for topic in soup.findAll('td',{'class':'topic'})]


def BeatNW(self):
    FORUM_URL = 'http//netwars.pl'
    

# r = session.get('http://netwars.pl')
# nw_time = parser.parse(r.headers['date']).replace(tzinfo=None)

In [211]:
topic = NwTopic.from_url(173454)
t = topic.soup.find_all('div', {'id':'topic_173454'})[0]
all_posts = t.findAll('div',class_='post')
collect_all_post = []
for post in all_posts:
    POST = {}
    POST['post_number'] = int(post['id'].split('_')[-1])
    POST['topic_number'] = 173454
    for subpost in post:
        if 'posthead' in subpost['class']:
            for sub_head in subpost.children:
                if 'p2_data' in sub_head['class']:
                    POST['date'] = sub_head.text
                if 'p2_nick' in sub_head['class']:
                    POST['user_href'] = sub_head.a['href']
                    POST['user_name'] = sub_head.a.text
                
        if 'post_body' in subpost['class']:
            POST['post_body'] = subpost.text
    collect_all_post.append(POST)
        


/home/i008/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [263]:
import pandas as pd

ids = topic.soup.findAll('div',{'class':'post'})
dates = topic.soup.findAll('div',{'class':'p2_data'})
nicks = topic.soup.findAll('div',{'class':'p2_nick'})
bodies = topic.soup.findAll('div',{'class':'post_body'})

dates = map(lambda x: x.text, dates)
post_bodies = map(lambda x: x.text, bodies)
user_hrefs = map(lambda x: x.a['href'], nicks)
user_names = map(lambda x: x.a.text, nicks)
ids = map(lambda x: x['id'].split('_')[-1], ids)

super_json_kurwo = [
    {
    'post_id':pid, 
    'post_date':pdate, 
    'user_href':href, 
    'user_name':uname, 
    'post_body':body
    } for pid,pdate,href,uname,body in 
     zip(ids, dates, user_hrefs, user_names, post_bodies)
]





In [9]:
r = requests.get('http://netwars.pl')
soup = BeautifulSoup(r.text)

/home/i008/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [256]:
POST


{'date': 'Dzisiaj, 15:43:26',
 'post_body': 'W 76 SPQR.Colos napisał: [Pokaż]\n\n\r\nCoś sie obawiam ze to nie twoja sprawa colos.',
 'post_number': 100,
 'topic_number': 173454,
 'user_href': '/user/28979',
 'user_name': 'ajnos'}

In [2]:
from apscheduler.schedulers.blocking import BlockingScheduler

In [44]:
from pytz import utc
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.jobstores.sqlalchemy import SQLAlchemyJobStore
from apscheduler.jobstores.redis import RedisJobStore
from apscheduler.executors.pool import ProcessPoolExecutor

jobstores = {
#     'mongo': {'type': 'mongodb'},
#     'default': SQLAlchemyJobStore(url='sqlite:///jobs.sqlite')
    'default': RedisJobStore()
}
executors = {
    'default': {'type': 'threadpool', 'max_workers': 20},
    'processpool': ProcessPoolExecutor(max_workers=5)
}

job_defaults = {
    'coalesce': False,
    'max_instances': 3
}
# scheduler = BlockingScheduler()
scheduler = BackgroundScheduler(jobstores=jobstores, executors=executors, job_defaults=job_defaults, timezone=utc)
import hashlib
import requests
L = []
def get_nw_hash():
    hash_string = hashlib.md5(requests.get('http://netwars.pl').text.encode('utf8')).hexdigest()
    
    if hash_string in L:
        L.append('oh now nw changed!')
    else:
        L.append(hash_string)    

scheduler.add_job(get_nw_hash, 'interval', seconds=30)
scheduler.start()

In [5]:
nw_job = scheduler.get_jobs()[0]

In [37]:
import requests
requests.get('http://localhost:8000')

<Response [200]>

In [8]:
import redis

In [9]:
r = redis.Redis()

In [16]:
r.lpush('ok909-0',['a','b'])

1

In [20]:
r.keys()

[b'ok909-0', b'ok', b'rq:queues', b'rq:finished:default']

In [35]:
r.set('kdjkdjkdjd','3202032020')

True

In [36]:
r.get('kdjkdjkdjd')

b'3202032020'

In [39]:
scheduler.shutdown()

In [52]:
scheduler.shutdown()